In [1]:
from postgres.postgres import *
from config.reader import *
from moex.moex import get_historical, cut_historical
import pandas as pd

In [2]:
cfg = read_config('config.json')

In [37]:
df = get_historical('SBER')
df.tail()

,TRADEDATE,CLOSE,VOLUME,VALUE,unix_time
2508,2023-03-07,174.01,55287380,9.555925e+09,1.678147e+09
2509,2023-03-09,172.55,80393110,1.394316e+10,1.678320e+09
2510,2023-03-10,172.53,50256970,8.636905e+09,1.678406e+09
2511,2023-03-13,172.15,74032280,1.270953e+10,1.678666e+09
2512,2023-03-14,175.71,89325670,1.560132e+10,1.678752e+09


In [39]:
conn = connect(cfg)

In [40]:
tickers = cfg.get("tickers")

In [41]:
cur = conn.cursor()
for t in tickers:
    cur.execute(f'create table {t}(TRADEDATE text, CLOSE float, VOLUME bigint, VALUE float, unix_time float)')
cur.execute("commit")
cur.close()
conn.close()

In [42]:
conn = connect(cfg)
for t in tickers:
    df = get_historical(t)
    for _, row in df.iterrows():
        insert(conn, t, "TRADEDATE, CLOSE, VOLUME, VALUE, unix_time", f'{row[0]},{row[1]}, {row[2]}, {row[3]}, {row[4]}')

In [43]:
conn.close()

In [45]:
conn = connect(cfg)
query(conn, "select * from SBER limit 10")

[('1985', 98.79, 593680, 59340002.8, 1364169600.0),
 ('1984', 97.2, 1283550, 126030358.8, 1364256000.0),
 ('1983', 96.75, 1261950, 121835900.2, 1364342400.0),
 ('1982', 98.59, 1971410, 192469794.9, 1364428800.0),
 ('1981', 98.76, 782000, 77268860.1, 1364515200.0),
 ('2008', 98.07, 2279650, 224932551.3, 1364774400.0),
 ('2007', 98.93, 3555020, 350934591.7, 1364860800.0),
 ('2006', 99.59, 3757240, 372483219.4, 1364947200.0),
 ('2005', 99.91, 5172930, 516688826.9, 1365033600.0),
 ('2004', 99.38, 5229210, 518651795.1, 1365120000.0)]

In [51]:
t = get_historical('SBER')
t["TRADEDATE"]

0       2013-03-25
1       2013-03-26
2       2013-03-27
3       2013-03-28
4       2013-03-29
           ...    
2508    2023-03-07
2509    2023-03-09
2510    2023-03-10
2511    2023-03-13
2512    2023-03-14
Name: TRADEDATE, Length: 2495, dtype: object